**Packages**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from datasets import load_dataset
import evaluate

#let's make longer output readable without horizontal scrolling
from pprint import pprint

/home/ubuntu/w266/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-26 23:00:27.808858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 23:00:32.112164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/C

**Necessary Functions**

In [2]:
rouge = evaluate.load('rouge')

**Data**

In [8]:
dataset = load_dataset("csebuetnlp/xlsum", "english")

Dataset xlsum downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csebuetnlp___xlsum/english/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 222.67it/s]


In [9]:
# EDA
len(dataset['train'])

306522

In [10]:
# EDA
dataset['train'][1]

{'id': 'uk-scotland-highlands-islands-11069985',
 'url': 'https://www.bbc.com/news/uk-scotland-highlands-islands-11069985',
 'title': 'Huge tidal turbine installed at Orkney test site',
 'summary': 'The massive tidal turbine AK1000 has been installed in 35m (114.8ft) of water at a test site in Orkney.',
 'text': 'Atlantis Resources unveiled the marine energy device at Invergordon ahead of it being shipped to Kirkwall. Trials on the device will now be run at the European Marine Energy Centre test site off Eday. The device stands 22.5m (73ft) tall, weighs 1,300 tonnes and has two sets of blades on a single unit. It could generate enough power for 1,000 homes.'}

In [14]:
index = pd.DataFrame({"index": list(range(len(dataset['train'])))})
sample_index = index.sample(n=2000, replace=False, random_state=1004)
sample_index[:5]

,index
235420,235420
172024,172024
253546,253546
224954,224954
214134,214134


In [30]:
id = []
url = []
title = []
article = []
summary = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    article.append(dataset["train"][i]['text'])

In [31]:
d = {'id': id, 'url': url, "title": title, 'article': article, 'summary': summary}
df = pd.DataFrame(data=d)
df.head(5)

,id,url,title,article,summary
0,uk-england-cornwall-55191422,https://www.bbc.com/news/uk-england-cornwall-5...,Care home manager: 'It felt like we were losin...,By Rebecca Ricks & Johnny O'SheaBBC Spotlight ...,"During the spring, at the height of the Covid-..."
1,uk-43893709,https://www.bbc.com/news/uk-43893709,Tafida Raqeeb: Who decides the care of sick ch...,"By Rachel SchraerBBC Reality Check So, why did...","The parents of five-year-old Tafida Raqeeb, wh..."
2,uk-politics-57050659,https://www.bbc.com/news/uk-politics-57050659,Labour reshuffle: Anneliese Dodds out in Starm...,Anneliese Dodds will now become the Labour Par...,Sir Keir Starmer has sacked his shadow chancel...
3,entertainment-arts-38221420,https://www.bbc.com/news/entertainment-arts-38...,Vinyl sales made more than downloads last week,By Mark SavageBBC Music reporter Vinyl sales m...,More money was spent on vinyl than downloaded ...
4,entertainment-arts-24046991,https://www.bbc.com/news/entertainment-arts-24...,Pirates of the Caribbean sequel delayed,Disney's Pirates of The Caribbean: Dead Men Te...,The next Pirates of the Caribbean film has bee...


**Baseline**

In [35]:
base_r1 = []
base_r2 = []
base_rL = []
base_rLs = []

for i in range(len(df['article'])):
    
    
    # first three sentences 
    candidate = ". ".join(df["article"][i].split('. ')[0:3]) + "."
    candidate = [candidate]
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references= ref)
    
    base_r1.append(results['rouge1'])
    base_r2.append(results['rouge2'])
    base_rL.append(results['rougeL'])
    base_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, 2000, 100):
        data = {'rouge1': base_r1, 'rouge2': base_r2, 'rogueL': base_rL, 'rogueLs': base_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'base_scores.csv', index=False)
        print(i)
        
data = {'rouge1': base_r1, 'rouge2': base_r2, 'rogueL': base_rL, 'rogueLs': base_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'base_scores.csv', index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [44]:
print("Average of all 2000")
print('rouge1 average: ', np.mean(base_r1))
print('rouge2 average: ', np.mean(base_r2))
print('rougeL average: ', np.mean(base_rL))
print('rougeLs average:', np.mean(base_rLs))

Average of all 2000
rouge1 average:  0.18615699708157343
rouge2 average:  0.02667181874604599
rougeL average:  0.12080542692742077
rougeLs average: 0.12080542692742077


In [46]:
print("Average of First 1000")
print('rouge1 average: ', np.mean(base_r1[:1000]))
print('rouge2 average: ', np.mean(base_r2[:1000]))
print('rougeL average: ', np.mean(base_rL[:1000]))
print('rougeLs average:', np.mean(base_rLs[:1000]))

Average of First 1000
rouge1 average:  0.18572856885833472
rouge2 average:  0.025796662812245914
rougeL average:  0.12019463217968686
rougeLs average: 0.12019463217968686


In [45]:
print("Average of Last 1000")
print('rouge1 average: ', np.mean(base_r1[1000:]))
print('rouge2 average: ', np.mean(base_r2[1000:]))
print('rougeL average: ', np.mean(base_rL[1000:]))
print('rougeLs average:', np.mean(base_rLs[1000:]))

Average of Last 1000
rouge1 average:  0.18658542530481215
rouge2 average:  0.027546974679846067
rougeL average:  0.12141622167515467
rougeLs average: 0.12141622167515467
